# Obtaining Your Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Understand the ETL process and the steps it consists of
* Understand the challenges of working with data from multiple sources 

## Task Description

You just got hired by Lego! Your first project is going to be to develop a pricing algorithm to help set a target price for new lego sets that are released to market. To do this, you're first going to need to start mining the company database in order to collect the information you need to develop a model.

Start by investigating the database stored in lego.db and joining the tables into a unified dataset!

> **Hint:** use this SQL query to preview the tables in an unknown database:
```sql
SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;
```

In [2]:
# Import needed packages
import pandas as pd
import numpy as np
import sqlite3

In [14]:
# Create a query-to-DataFrame function
def query_to_df(query):
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall())
    df.columns = [x[0] for x in cur.description]
    return df

In [4]:
# Set up database connection and cursor
conn = sqlite3.connect('lego.db')
cur = conn.cursor()

In [5]:
# View table names
cur.execute("""
    SELECT name
    FROM sqlite_master
    WHERE type = 'table'
    ORDER BY name;""").fetchall()


[('product_details',),
 ('product_info',),
 ('product_pricing',),
 ('product_reviews',)]

## View the tables

In [24]:
# Query first table to see column names
product_details = query_to_df("""SELECT * FROM product_details""")
display(product_details.head())
print(len(product_details))

,prod_id,prod_desc,prod_long_desc,theme_name
0,630,blah,blahblah,Classic
1,2304,blah,blahblah,DUPLO®
2,7280,blah,blahblah,City
3,7281,blah,blahblah,City
4,7499,blah,blahblah,City


744


In [21]:
print(product_details.prod_desc.unique())
print(product_details.prod_long_desc.unique())

['blah']
['blahblah']


In [25]:
product_info = query_to_df("""SELECT * FROM product_info""")
display(product_info.head())
print(len(product_info))

,prod_id,ages,piece_count,set_name
0,630,4+,1,Brick Separator
1,2304,1½-5,1,LEGO® DUPLO® Green Baseplate
2,7280,5-12,2,Straight & Crossroad Plates
3,7281,5-12,2,T-Junction & Curved Road Plates
4,7499,5-12,24,Flexible and Straight Tracks


744


In [26]:
product_pricing = query_to_df("""SELECT * FROM product_pricing""")
display(product_pricing.head())
print(len(product_pricing))

,prod_id,country,list_price
0,75823,US,29.99
1,75822,US,19.99
2,75821,US,12.99
3,21030,US,99.99
4,21035,US,79.99


10870


In [27]:
product_reviews = query_to_df("""SELECT * FROM product_reviews""")
display(product_reviews.head())
print(len(product_reviews))

,prod_id,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,630,180.0,4.0,Very Easy,4.8,4.6
1,2304,15.0,4.4,Easy,4.0,3.3
2,7280,85.0,4.1,Very Easy,3.5,2.3
3,7281,40.0,4.1,Very Easy,3.9,2.8
4,7499,89.0,2.9,Easy,2.5,2.2


744


## Build the big query

In [28]:
df = query_to_df("""
    SELECT *
    FROM product_details
    JOIN product_info
    USING(prod_id)
    JOIN product_pricing
    USING(prod_id)
    JOIN product_reviews
    USING(prod_id);""")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Data columns (total 14 columns):
prod_id              10870 non-null int64
prod_desc            10870 non-null object
prod_long_desc       10870 non-null object
theme_name           10870 non-null object
ages                 10870 non-null object
piece_count          10870 non-null int64
set_name             10870 non-null object
country              10870 non-null object
list_price           10870 non-null object
num_reviews          9449 non-null float64
play_star_rating     9321 non-null float64
review_difficulty    10870 non-null object
star_rating          9449 non-null float64
val_star_rating      9301 non-null float64
dtypes: float64(4), int64(2), object(8)
memory usage: 1.2+ MB


In [29]:
df.describe()

,prod_id,piece_count,num_reviews,play_star_rating,star_rating,val_star_rating
count,1.087000e+04,10870.000000,9449.000000,9321.000000,9449.000000,9301.000000
mean,6.181634e+04,503.936431,17.813737,4.355413,4.510319,4.214439
std,1.736390e+05,831.209318,38.166693,0.617272,0.516463,0.670906
min,6.300000e+02,1.000000,1.000000,1.000000,1.800000,1.000000
25%,2.112300e+04,97.000000,2.000000,4.000000,4.300000,4.000000
50%,4.207350e+04,223.000000,6.000000,4.500000,4.600000,4.300000
75%,7.124800e+04,556.000000,14.000000,4.800000,5.000000,4.700000
max,2.000431e+06,7541.000000,367.000000,5.000000,5.000000,5.000000


In [30]:
df.to_csv('Lego_data_full.csv')

## Summary
Nice work! You're working more and more independently through the workflow and ensuring data integrity!